In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define a function to create the model with specified hyperparameters
def create_model(epochs, hidden_layer_size, activation):
    img_height, img_width = 224, 224
    batch_size = 32

    # Load VGG16 with pre-trained weights
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

    # Freeze all layers in the base model
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers with convolutional layers and max-pooling
    model = Sequential([
        base_model,

        GlobalAveragePooling2D(),

        Dense(hidden_layer_size, activation=activation),
        Dense(2, activation='softmax')  # Softmax for multi-class classification
    ])

    # Data Generators
    train_datagen = ImageDataGenerator(rescale=1./255)
    train_gen = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/DataSet Grad Project/dataset_1/training',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )

    val_datagen = ImageDataGenerator(rescale=1./255)
    val_gen = val_datagen.flow_from_directory(
        '/content/drive/MyDrive/DataSet Grad Project/dataset_1/validation',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )

    # Compile the model with categorical cross-entropy and the metric 'accuracy'
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Training the model
    history = model.fit(train_gen, epochs=epochs, validation_data=val_gen)

    # Evaluate the model
    evaluation = model.evaluate(val_gen)
    val_accuracy = evaluation[1]
    print(f"Last Validation Accuracy: {val_accuracy}")

    return history, val_accuracy,val_gen,train_gen,model

# Define the hyperparameter values for the grid search
epochs_values = [5, 10, 15]
hidden_layer_sizes = [64, 128, 256]
activations = ['relu', 'tanh', 'sigmoid']




In [ ]:
# Define an empty list to store the results
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
results = []

# Initialize best_score and best_params
best_score = 0.0
best_params = {}


# Perform grid search
for epochs in epochs_values:
    for hidden_layer_size in hidden_layer_sizes:
        for activation in activations:
            print(f"Training with: Epochs={epochs}, Hidden Layer Size={hidden_layer_size}, Activation={activation}")
            history, val_accuracy,val_gen,train_gen,model = create_model(epochs, hidden_layer_size, activation)
            class_labels = train_gen.class_indices

            class_labels = {v: k for k, v in class_labels.items()}

            true_labels = []
            predicted_classes = []

            for i in range(len(val_gen)):
                batch_images, batch_labels = next(val_gen)
                true_labels.extend(batch_labels.argmax(axis=1))
                predicted_classes.extend(model.predict(batch_images).argmax(axis=1))

            # Calculate the F1 score, precision, recall, and confusion matrix
            f1_score_value = f1_score(true_labels, predicted_classes, average='weighted')
            precision = precision_score(true_labels, predicted_classes, average='weighted')
            recall = recall_score(true_labels, predicted_classes, average='weighted')
            conf_matrix = confusion_matrix(true_labels, predicted_classes)

            print("F1 Score:", f1_score_value)
            print("Precision:", precision)
            print("Recall:", recall)
            print("Confusion Matrix:\n", conf_matrix)

            # Generate a classification report
            report = classification_report(true_labels, predicted_classes, target_names=class_labels.values())

            print("Classification Report:\n", report)


            result = {
                'Epochs': epochs,
                'Hidden Layer Size': hidden_layer_size,
                'Activation': activation,
                'Validation Accuracy': val_accuracy
            }
            results.append(result)

            # Update best hyperparameters and best score
            if val_accuracy > best_score:
                best_score = val_accuracy
                best_params = {
                    'epochs': epochs,
                    'hidden_layer_size': hidden_layer_size,
                    'activation': activation
                }


Training with: Epochs=5, Hidden Layer Size=64, Activation=relu
94765736/94765736 [==============================] - 1s 0us/step
Found 1840 images belonging to 2 classes.
Found 460 images belonging to 2 classes.
Epoch 1/5
58/58 [==============================] - 394s 7s/step - loss: 0.6897 - accuracy: 0.5505 - val_loss: 0.6700 - val_accuracy: 0.5565
Epoch 2/5
58/58 [==============================] - 8s 139ms/step - loss: 0.6483 - accuracy: 0.6136 - val_loss: 0.6651 - val_accuracy: 0.5543
Epoch 3/5
58/58 [==============================] - 8s 142ms/step - loss: 0.6396 - accuracy: 0.6332 - val_loss: 0.6465 - val_accuracy: 0.5848
Epoch 4/5
58/58 [==============================] - 8s 143ms/step - loss: 0.6437 - accuracy: 0.6332 - val_loss: 0.6248 - val_accuracy: 0.6348
Epoch 5/5
15/15 [==============================] - 2s 107ms/step - loss: 0.6175 - accuracy: 0.6391
Last Validation Accuracy: 0.6391304135322571
1/1 [==============================] - 1s 944ms/step
F1 Score: 0.6240942028985508


In [ ]:
# Print the best hyperparameters and the best validation accuracy
print("Best Hyperparameters: ", best_params)
print("Best Validation Accuracy: ", best_score)

# Create a DataFrame from the results list
df = pd.DataFrame(results)

# Group the DataFrame by the 'Activation' column
grouped = df.groupby('Activation')

# Print the grouped DataFrame
for name, group in grouped:
    print(name)
    print(group)


Best Hyperparameters:  {'epochs': 15, 'hidden_layer_size': 128, 'activation': 'tanh'}
Best Validation Accuracy:  0.739130437374115
relu
    Epochs  Hidden Layer Size Activation  Validation Accuracy
0        5                 64       relu             0.639130
3        5                128       relu             0.654348
6        5                256       relu             0.684783
9       10                 64       relu             0.667391
12      10                128       relu             0.654348
15      10                256       relu             0.700000
18      15                 64       relu             0.728261
21      15                128       relu             0.732609
24      15                256       relu             0.713043
sigmoid
    Epochs  Hidden Layer Size Activation  Validation Accuracy
2        5                 64    sigmoid             0.647826
5        5                128    sigmoid             0.682609
8        5                256    sigmoid          